
# ViEWS 3 constituent models 
## ViEWS example notebook, cm level



## Importing modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Basics
import numpy as np
import pandas as pd

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

import os
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from FetchData import FetchData, RetrieveFromList, document_queryset, ReturnQsList, document_ensemble
from ViewsEstimators import *


## Common parameters

In [ ]:
dev_id = 'onboard001'
run_id = dev_id

# Generating a new run if necessary

try:
    ViewsMetadata().new_run(name=run_id,description='onboarding',min_month=1,max_month=999)
except KeyError:
    pass

RerunQuerysets = True

EndOfHistory = 517
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

# Specifying partitions
calib_partitioner_dict = {"train":(121,408),"predict":(409,456)}
test_partitioner_dict = {"train":(121,456),"predict":(457,504)}
future_partitioner_dict = {"train":(121,504),"predict":(505,516)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

# Retrieve data

In [ ]:
level = 'cm'
qslist = ReturnQsList(level,new_qs=True)

In [ ]:
from FetchData import fetch_cm_data_from_model_def

Datasets=fetch_cm_data_from_model_def(qslist, EndOfHistory)

# Generating predictions
Using the ViEWS3 partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome.

# Specify models in ensemble

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels('cm')
    
for imodel,model in enumerate(ModelList):
    print(imodel, model['modelname'], model['data_train'])

In [ ]:
for ds in Datasets:
    df = ds['df']
#    print(ds['Name'],df.isna().sum())
    ds['df']=df.fillna(0)

In [ ]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = False
force_rewrite = True
force_retrain = True

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList[:1]:
    if 'Markov' not in model['modelname']:
        
        modelstore = storage.Storage()
#        ct = datetime.now()
        print(i, model['modelname'])
        print('Calibration partition')
        model['Algorithm_text'] = str(model['algorithm'])
        model['RunResult_calib'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "calib",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_calib',
                author_name        = "JED",
        )

    #    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
#        ct = datetime.now()
        if force_rewrite:
            print(model['predstore_calib'], ', run',  run_id, 'force_rewrite=True, predicting')
            predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'],overwrite=True)
        else:
            print('Trying to retrieve predictions', ct)
            try:
                predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
            except KeyError:
                print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
                predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
                predictions_calib.forecasts.set_run(run_id)
                predictions_calib.forecasts.to_store(name=model['predstore_calib'])

#        ct = datetime.now()
        print('Test partition')
        modelstore = storage.Storage()
        model['RunResult_test'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":test_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_test',
                author_name        = "JED",
        )
#        ct = datetime.now()
        
        if force_rewrite:
            print(model['predstore_test'], ', run',  run_id, 'force_rewrite=True, predicting')
            predictions_test = model['RunResult_test'].run.predict("test","predict", model['RunResult_test'].data)
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'],overwrite=True)
        else:
            print('Trying to retrieve predictions', ct)
    #    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
            try:
                predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
            except KeyError:
                print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
                predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
                predictions_test.forecasts.set_run(run_id)
                predictions_test.forecasts.to_store(name=model['predstore_test'])

# Predictions for true future
        if includeFuture:
#            ct = datetime.now()
            print('Future')
            modelstore = storage.Storage()
            model['RunResult_future'] = RunResult.retrain_or_retrieve(
                    retrain            = force_retrain,
                    store              = modelstore,
                    partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                    stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                    dataset            = RetrieveFromList(Datasets,model['data_train']),
                    queryset_name      = model['queryset'],
                    partition_name     = "test",
                    timespan_name      = "train",
                    storage_name       = model['modelname'] + '_future',
                    author_name        = "JED",
            )
#            ct = datetime.now()
            
            if force_rewrite:
                print(model['predstore_future'], ', run',  run_id, 'force_rewrite=True, predicting')
                predictions_future = model['RunResult_future'].run.predict(EndOfHistory, model['RunResult_future'].data)
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'],overwrite=True)
            else:
                print('Trying to retrieve predictions', ct)
                model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(EndOfHistory)
                predictions_future.forecasts.to_store(name=model['predstore_future'])  
                
    
    i = i + 1

print('All done')